In [1]:
# source : https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [2]:
#Loading a CSV file is straightforward with Spark csv packages.
#sc =SparkContext()
sqlContext = SQLContext(sc)
data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('FileStore/tables/train.csv')

In [3]:
#Remove the columns we do not need and have a look the first five rows:
drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
data = data.select([column for column in data.columns if column not in drop_list])
data.show(5)

+--------------+--------------------+
 Category| Descript|
+--------------+--------------------+
 WARRANTS| WARRANT ARREST|
OTHER OFFENSES|TRAFFIC VIOLATION...|
OTHER OFFENSES|TRAFFIC VIOLATION...|
 LARCENY/THEFT|GRAND THEFT FROM ...|
 LARCENY/THEFT|GRAND THEFT FROM ...|
+--------------+--------------------+
only showing top 5 rows

In [4]:
#Apply printSchema() on the data which will print the schema in a tree format:
data.printSchema()

root
-- Category: string (nullable = true)
-- Descript: string (nullable = true)

In [5]:
#Top 20 crime categories:
from pyspark.sql.functions import col
data.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
 Category| count|
+--------------------+------+
 LARCENY/THEFT|174900|
 OTHER OFFENSES|126182|
 NON-CRIMINAL| 92304|
 ASSAULT| 76876|
 DRUG/NARCOTIC| 53971|
 VEHICLE THEFT| 53781|
 VANDALISM| 44725|
 WARRANTS| 42214|
 BURGLARY| 36755|
 SUSPICIOUS OCC| 31414|
 MISSING PERSON| 25989|
 ROBBERY| 23000|
 FRAUD| 16679|
FORGERY/COUNTERFE...| 10609|
 SECONDARY CODES| 9985|
 WEAPON LAWS| 8555|
 PROSTITUTION| 7484|
 TRESPASS| 7326|
 STOLEN PROPERTY| 4540|
SEX OFFENSES FORC...| 4388|
+--------------------+------+
only showing top 20 rows

## Model Pipeline
Spark Machine Learning Pipelines API is similar to Scikit-Learn. Our pipeline includes three steps:

1. **regexTokenizer**: Tokenization (with Regular Expression) 
2. **stopwordsRemover**: Remove Stop Words
3. **countVectors**: Count vectors (“document-term vectors”)

In [7]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="Descript", outputCol="words", pattern="\\W")
# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

## StringIndexer
StringIndexer encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In our case, the label column (Category) will be encoded to label indices, from 0 to 32; the most frequent label (LARCENY/THEFT) will be indexed as 0.

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

+--------------+--------------------+--------------------+--------------------+--------------------+-----+
 Category| Descript| words| filtered| features|label|
+--------------+--------------------+--------------------+--------------------+--------------------+-----+
 WARRANTS| WARRANT ARREST| [warrant, arrest]| [warrant, arrest]|(809,[17,32],[1.0...| 7.0|
OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(809,[11,17,35],[...| 1.0|
OTHER OFFENSES|TRAFFIC VIOLATION...|[traffic, violati...|[traffic, violati...|(809,[11,17,35],[...| 1.0|
 LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, fr...|(809,[0,2,3,4,6],...| 0.0|
 LARCENY/THEFT|GRAND THEFT FROM ...|[grand, theft, fr...|[grand, theft, fr...|(809,[0,2,3,4,6],...| 0.0|
+--------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [10]:
dataset.select(['features']).head(2)


Out[ 33 ]: 
[Row(features=SparseVector(809, {17: 1.0, 32: 1.0})),
 Row(features=SparseVector(809, {11: 1.0, 17: 1.0, 35: 1.0}))]

## Partition Training & Test sets

In [12]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 614666
Test Dataset Count: 263383

##Model Training and Evaluation

**Logistic Regression using Count Vector Features**

Our model will make predictions and score on the test set; we then look at the top 10 predictions from the highest probability.

In [14]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)


+------------------------------+-------------+------------------------------+-----+----------+
 Descript| Category| probability|label|prediction|
+------------------------------+-------------+------------------------------+-----+----------+
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8711581002180206,0.02115...| 0.0| 0.0|
+------------------------------+-------------+------------------------------+-----+----------+
only showing top 10 rows

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 36 ]: 0.9725282146509521

**Logistic Regression using TF-IDF Features**

In [17]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+-------------+------------------------------+-----+----------+
 Descript| Category| probability|label|prediction|
+------------------------------+-------------+------------------------------+-----+----------+
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
THEFT, BICYCLE, <$50, NO SE...|LARCENY/THEFT|[0.8730538827818622,0.02078...| 0.0| 0.0|
+------------------------------+-------------+------------------------------+-----+----------+
only showing top 10 rows

In [18]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 38 ]: 0.9722954523853476

**The result is the same.**

**Cross-Validation**

Let’s now try cross-validation to tune our hyper parameters, and we will only tune the count vectors Logistic Regression.

In [21]:
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 39 ]: 0.9919124901837848

**Naive Bayes**

In [23]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+----------------------------+-------------+------------------------------+-----+----------+
 Descript| Category| probability|label|prediction|
+----------------------------+-------------+------------------------------+-----+----------+
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
PETTY THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.9999999999837754,1.54986...| 0.0| 0.0|
+----------------------------+-------------+------------------------------+-----+----------+
only showing top 10 rows

In [24]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 41 ]: 0.9939847228864086

**Random Forest**

In [26]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Descript","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+----------------------------+-------------+------------------------------+-----+----------+
 Descript| Category| probability|label|prediction|
+----------------------------+-------------+------------------------------+-----+----------+
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
GRAND THEFT FROM LOCKED AUTO|LARCENY/THEFT|[0.6402649319141156,0.06141...| 0.0| 0.0|
+----------------------------+-------------+------------------------------+-----+----------+
only showing top 10 rows

In [27]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[ 43 ]: 0.6490477620735787

Random forest is a very good, robust and versatile method, however it’s no mystery that for high-dimensional sparse data it’s not a best choice.

**随机森林是一种非常好的，强大而通用的方法，但对于高维稀疏数据来说，它并不是最好的选择。**

It is obvious that Logistic Regression will be our model in this experiment, with cross validation.

**很明显Logistic回归将是我们在这个实验中的模型，具有交叉验证。**

This brings us to the end of the article. Source code that create this post can be found on Github. I look forward to hear any feedback or questions.